In [1]:
import pandas as pd
from pulp import LpMinimize, LpMaximize, LpProblem, LpVariable

In [2]:
cnap = pd.read_csv('core_nutrient_amounts_prices_v3.csv', index_col=0)
cnap = cnap.rename(columns={'name': 'nutrient_name'})
constraints = pd.read_csv('nutrient_constraints_csv.csv').set_index('nutrient_nbr').drop(columns=['id', 'rank']).rename(columns={'name': 'nutrient_name'})
constraints = constraints.loc[constraints.target.notna() | constraints.ll.notna() | constraints.ul.notna()]    
n_values = cnap.pivot(index='nutrient_nbr', columns='food', values='nutrient_unit_per_dollar')
cc = constraints.join(n_values, how='left')
first_food_idx = cc.columns.get_loc('Almonds')
cc.iloc[:, first_food_idx:] = cc.iloc[:, first_food_idx:].fillna(0)

In [13]:
prob = LpProblem("Minimize_Cost", LpMinimize)
decision_variables = []
first_food_idx = cc.columns.get_loc('Almonds')
prob += sum([decision_variable for decision_variable in decision_variables])
for food in cc.columns[first_food_idx:]:
    decision_variables.append(LpVariable(f"{food}", lowBound=0))
for i, row in cc.iterrows():
    constraint = 0
    for j, food in enumerate(cc.columns[first_food_idx:]):
        constraint += decision_variables[j] * row[food]
    if pd.notna(row.target):
        print(row.nutrient_name, row.target)
        prob += constraint == row.target
    elif pd.notna(row.ll):
        prob += constraint >= row.ll
    elif pd.notna(row.ul):
        prob += constraint <= row.ul

Energy 3500.0


In [14]:
prob.solve()
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)

Beef_Liver = 0.030385023
Chia_seeds = 1.3500218
Eggs = 0.057788012
Mussels = 7.4565052
Potatoes = 0.581872
Salmon = 2.4855609
Sunflower_seeds = 1.5915374
Tomatoes = 4.1993127
__dummy = None


In [5]:
fs = pd.DataFrame([(v.name, v.varValue) for v in prob.variables() if v.varValue != 0], columns=['food', 'spend']).iloc[:-1,:]
cn = cc.set_index('nutrient_name').iloc[:, 4:]
# Rank the foods for each nutrient
food_ranks = cn.rank(axis=1, method='min', ascending=False)
final_foods = cnap.loc[cnap.food.isin([v.name for v in prob.variables() if v.varValue != 0])].drop_duplicates('food')

In [6]:
ffs = final_foods.merge(fs, on='food')

In [7]:
ffs.spend / ffs.price_per_100_g

0    24.577737
1     4.809704
2     1.409288
3     3.382211
4     0.174748
dtype: float64

In [8]:
ffs.spend.sum() * 365

5395.079166379999

In [9]:
14 * 100 / 453.592

3.086474188257289

In [10]:
# TODO: add fiber constraint, maybe variety constraint, check numbers so that oysters probably don't show up

In [11]:
food_ranks['Brown rice'].sort_values()

nutrient_name
Caffeine                           1.0
Sodium, Na                         2.0
Thiamin                            5.0
Niacin                             5.0
Magnesium, Mg                      5.0
Phosphorus, P                      6.0
Lycopene                           6.0
Vitamin B-6                        7.0
Carbohydrate, by difference        8.0
Energy                             8.0
Riboflavin                         8.0
Zinc, Zn                           8.0
Selenium, Se                      10.0
PUFA 20:5 n-3 (EPA)               11.0
Vitamin D (D2 + D3)               12.0
Fiber, total dietary              12.0
PUFA 22:6 n-3 (DHA)               12.0
Iron, Fe                          15.0
Choline, total                    15.0
Copper, Cu                        15.0
PUFA 22:5 n-3 (DPA)               15.0
Protein                           16.0
Potassium, K                      18.0
PUFA 18:2                         18.0
Retinol                           22.0
Vitamin B-1